In [1]:
from sklearn.datasets import load_iris
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# loading raw data
raw_data = load_iris()

# separating data and tareget
data = raw_data.data
target = raw_data.target

In [3]:
# separating train and test datas
trainData, testData, trainTarget, testTarget = train_test_split(data, target, test_size=.2, random_state=42)

In [4]:
# define different k for KMeans
# the member 120 is added to the list to show the effect 
# of different verification methods.
K = [2, 3, 4, 5, 6, 7, 8, 120]

In [5]:
# creating a pandas DF from train data to use pandas features
trainDF = pd.DataFrame(trainData)
trainDF['target'] = trainTarget
# creating a pandas DF from train data to use pandas features
testDF = pd.DataFrame(testData)
testDF['target'] = testTarget

# NOTE: to see the structure of the DF uncomment the following line
# print(trainDF)

## what we are going to do

here we are gonna fit the train data with kmeans and after that find the most label in each cluster and update the label that kmeans has assign to that cluster, after that testing the accuracy rate using the test data(20% of the raw data).

what we expect is to see that the rate that we calculate is not a good measure for finding K.

In [10]:
# fitting train data with different k
# and testing on test datas
for k in K:
#     creating an object from Kmeans and passing k
    kmeans = KMeans(n_clusters=k)
#     fitting
    kmeans.fit(trainData)
#     creating a temp DF for train data and adding labels to it
    tempDF = trainDF.loc[:,:]
    tempDF['trained'] = kmeans.labels_
    
#     finding different uniqe labels added to datas by kmeans
    trainedLbl = tempDF['trained'].unique()
    
#     creating a temp DF for test datas and adding predicted labels to it
    temp = testDF.loc[:,:]
    temp['predicted'] = kmeans.predict(testData)
    
#     the following counter is used to count the number of test datas
#     that are labeled correctly
    count = 0
    for lbl in trainedLbl:
#         getting the most label in each cluster and change the label added by kmeans
#         to this value(the most label is real label of cluster)
        realLbl = int(tempDF[tempDF['trained'] == lbl]['target'].mode()[0])
        t1 = temp[temp['predicted'] == lbl]
        
        t1['predicted'] = realLbl
    
#         we count the number of datas that their target label is equal to predicted label
        count += t1[t1['target'] == t1['predicted']].shape[0]

#     calculating the rate of prediction for each k
    print(f'k= {k}, rate= {count / temp.shape[0]}')
        

k= 2, rate= 0.7
k= 3, rate= 0.9666666666666667
k= 4, rate= 0.8333333333333334
k= 5, rate= 0.9666666666666667
k= 6, rate= 0.9666666666666667
k= 7, rate= 0.9666666666666667
k= 8, rate= 0.9666666666666667
k= 120, rate= 0.9666666666666667


## explain result

as it is clear from the output of previous cell, the rate for several values of K is equal, so we can't say which one is the best K for this data set. that's because the method we used does not consider the number of clusters
jsut considers the datas that have been clustered correctly in one cluster.

# سوال دوم تمرین (بررسی بهینه بودن بدون داشتن داده تست)

for this task we can use RAND INDEX method.
this method works fine with kmeans and also can be used to find the best k.

#    Rand Index

In [7]:
for k in K:
#     creating an object from Kmeans and passing k
    kmeans = KMeans(n_clusters=k)
#     fitting
    kmeans.fit(trainData)
#     creating a temp DF for train data and adding labels to it
    tempDF = trainDF.loc[:,:]
    tempDF['trained'] = kmeans.labels_
#     printig rand index rate for each k
    print(f"k= {k}, rate= {adjusted_rand_score(tempDF['target'], tempDF['trained'])}")
    

k= 2, rate= 0.5310402626123087
k= 3, rate= 0.6788174668657071
k= 4, rate= 0.6015631698168469
k= 5, rate= 0.6004933898120142
k= 6, rate= 0.42428681033682414
k= 7, rate= 0.4706851679001883
k= 8, rate= 0.42723794077718685
k= 120, rate= 0.0005741392786039861


## compair two methods that are used

but rand index considers the number of clusters indirectly, because it counts the number of pairs that are clustered correctly. the pairs with the same label should be in the same cluster to be enumerated, also
the pairs that have different labels should be in different clusters to be enumerated so, in this method with consider the number of cluster